In [1]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [17]:
jpy = pd.read_csv('jpy-datasets.csv', index_col='Date')
print(f'Number of rows and colums: {jpy.shape}')
jpy.head()

Number of rows and colums: (1828, 4)


,Price,Open,High,Low
Date,,,,
2014-01-01,115.64,115.48,115.71,115.33
2014-01-02,116.07,115.52,116.45,115.19
2014-01-03,116.07,115.90,117.58,115.84
2014-01-06,116.87,115.91,117.48,115.84
2014-01-07,116.97,116.75,117.64,116.43


In [18]:
usd = pd.read_csv('usd-datasets.csv', index_col='Date')
print(f'Number of rows and colums: {usd.shape}')
usd.head()

Number of rows and colums: (1809, 4)


,Price,Open,High,Low
Date,,,,
2014-01-01,12170.0,12170.0,12170.0,12170.0
2014-01-02,12160.0,12195.0,12260.0,12150.0
2014-01-03,12170.0,12160.0,12245.0,12160.0
2014-01-06,12180.0,12195.0,12245.0,12180.0
2014-01-07,12237.5,12197.5,12277.5,12197.5


In [4]:
## training config

In [19]:
jpy_training_set = jpy.iloc[:round(0.8*len(jpy))].values
jpy_test_set = jpy.iloc[round(0.8*len(jpy)):].values

In [20]:
jpy_training_set

array([[115.64, 115.48, 115.71, 115.33],
       [116.07, 115.52, 116.45, 115.19],
       [116.07, 115.9 , 117.58, 115.84],
       ...,
       [133.97, 134.54, 135.41, 133.35],
       [133.83, 134.01, 134.93, 133.64],
       [133.94, 133.78, 134.34, 133.63]])

In [21]:
jpy_test_set

array([[134.28, 133.97, 134.82, 133.73],
       [135.29, 134.4 , 135.63, 134.22],
       [134.13, 135.29, 136.3 , 133.84],
       ...,
       [136.05, 136.41, 136.76, 135.92],
       [136.13, 136.24, 136.41, 135.99],
       [137.19, 137.06, 137.7 , 137.06]])

In [13]:
## Feature Scaling

In [24]:
sc = MinMaxScaler(feature_range=(-1,1))
jpy_training_set_scaled = sc.fit_transform(jpy_training_set)
jpy_test_set_scaled = sc.fit_transform(jpy_test_set)

In [25]:
jpy_training_set_scaled

array([[-0.17380063, -0.17978173, -0.25730659, -0.16498122],
       [-0.14909509, -0.1774842 , -0.21489971, -0.17307137],
       [-0.14909509, -0.15565767, -0.15014327, -0.13550997],
       ...,
       [ 0.87934502,  0.91499138,  0.87163324,  0.87633632],
       [ 0.87130135,  0.88454911,  0.84412607,  0.89309448],
       [ 0.87762137,  0.87133831,  0.81031519,  0.89251661]])

In [26]:
jpy_test_set_scaled

array([[-0.22983355, -0.24928   , -0.26980039, -0.23397224],
       [-0.16517286, -0.22176   , -0.21764327, -0.20158625],
       [-0.23943662, -0.1648    , -0.17450097, -0.22670192],
       ...,
       [-0.11651729, -0.09312   , -0.14488088, -0.0892267 ],
       [-0.11139565, -0.104     , -0.1674179 , -0.08460013],
       [-0.04353393, -0.05152   , -0.08435287, -0.01387971]])

In [42]:
# Creating a data structure with 60 time-steps and 1 output
jpy_X_train = []
jpy_y_train = []
for i in range(60, len(jpy_training_set)):
    jpy_X_train.append(jpy_training_set_scaled[i-60:i, 0])
    jpy_y_train.append(jpy_training_set_scaled[i, 0])
jpy_X_train, jpy_y_train = np.array(jpy_X_train), np.array(jpy_y_train)
jpy_X_train = np.reshape(jpy_X_train, (jpy_X_train.shape[0], jpy_X_train.shape[1], 1))
#(740, 60, 1)


In [47]:
# Generate model lstm

In [60]:
model = Sequential()

In [61]:
#Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True, input_shape = (jpy_X_train.shape[1], 1)))
model.add(Dropout(0.2))
# Adding a second LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
# Adding a fourth LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50))
model.add(Dropout(0.2))
# Adding the output layer
model.add(Dense(units = 1))

In [62]:
# Compiling the RNN
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
model.fit(jpy_X_train, jpy_y_train, epochs = 100, batch_size = 32)


Epoch 1/100
1402/1402 [==============================] - 7s 5ms/step - loss: 0.0387
Epoch 2/100
1402/1402 [==============================] - 6s 4ms/step - loss: 0.0189
Epoch 3/100
1402/1402 [==============================] - 6s 4ms/step - loss: 0.0169
Epoch 4/100
1402/1402 [==============================] - 6s 4ms/step - loss: 0.0155
Epoch 5/100
1402/1402 [==============================] - 6s 4ms/step - loss: 0.0142
Epoch 6/100
1402/1402 [==============================] - 6s 4ms/step - loss: 0.0145
Epoch 7/100
1402/1402 [==============================] - 6s 4ms/step - loss: 0.0123
Epoch 8/100
1402/1402 [==============================] - 6s 4ms/step - loss: 0.0117
Epoch 9/100
1402/1402 [==============================] - 6s 4ms/step - loss: 0.0109
Epoch 10/100
1402/1402 [==============================] - 6s 4ms/step - loss: 0.0106
Epoch 11/100
1402/1402 [==============================] - 6s 4ms/step - loss: 0.0108
Epoch 12/100
1402/1402 [==============================] - 6s 4ms/step - l

In [65]:
model_json=model.to_json()
with open("blueprint_model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights('blueprint_model.h5')

In [67]:
model.save('model.h5')

In [71]:
model.save('model.h5py')

In [73]:
# Creating a data structure with 60 time-steps and 1 output
jpy_X_test = []
jpy_y_test = []
for i in range(60, len(jpy_test_set)):
    jpy_X_test.append(jpy_test_set_scaled[i-60:i, 0])
    jpy_y_test.append(jpy_test_set_scaled[i, 0])
jpy_X_test, jpy_y_test = np.array(jpy_X_test), np.array(jpy_y_test)
jpy_X_test = np.reshape(jpy_X_test, (jpy_X_test.shape[0], jpy_X_test.shape[1], 1))
#(740, 60, 1)


In [74]:
scores = model.evaluate(jpy_X_test, jpy_y_test, verbose=0)

In [77]:
model.metrics_names, scores*100

(['loss'], 0.5305555023274269)

In [78]:
scores = model.evaluate(jpy_X_train, jpy_y_train, verbose=0)

In [80]:
scores*100

0.21895256322594664